In [1]:
#pip install nibabel
#pip install topology_radiomics

In [14]:
#pip install topology_radiomics

In [15]:
#pip install openpyxl

In [1]:
#pip install numpy==1.19

In [14]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 10 10:10:58 2020

@author: hyeminum / gpinedaortiz
"""
import glob
import logging
import topology_radiomics as toprad
import nibabel as nib
import os
import numpy as np
import pandas as pd
import skimage as ski
import itertools  
from itertools import zip_longest 

from skimage import data, util
from skimage.measure import label, regionprops

FORMAT = '%(asctime)-15s %(levelname)s %(funcName)s  %(message)s'
logging.basicConfig(format=FORMAT, level=logging.DEBUG)


## Set directories:  Local Shape and topology features (provide multilabel tumor mask (nii.gz) as input)

In [15]:
root_path='/app/Data/_Brain/Radiology/_Adult/_Glioma/'

In [29]:
mri_data='xCures'
time_point='Baseline'
input_path= os.path.join(root_path, mri_data,'Preprocessed/Pre-op-Post-op/*/'+ time_point ,'*brats_SRI24.nii.gz')
output_path =os.path.join(root_path, mri_data,'Feature_extraction/Radiomics_features/')
print('input_path...', input_path)
print('output_path...',output_path)
labels = glob.glob(input_path)

labels = sorted(labels)
print(len(labels))
labels

input_path... /app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/*/Baseline/*brats_SRI24.nii.gz
output_path... /app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Feature_extraction/Radiomics_features/
13


['/app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/100_0551/Baseline/100_0551_01_tumor_segmentation_brats_SRI24.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/100_1308/Baseline/100_1308_01_tumor_segmentation_brats_SRI24.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/100_296/Baseline/100_296_01_tumor_segmentation_brats_SRI24.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/100_302/Baseline/100_302_01_tumor_segmentation_brats_SRI24.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/100_4660/Baseline/100_4660_01_tumor_segmentation_brats_SRI24.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/100_5980/Baseline/100_5980_01_tumor_segmentation_brats_SRI24.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/100_719/Baseline/100_719_01_tum

In [ ]:
# Read list of patients from a list CSV XLSX

# 
import pandas as pd 
xlsx_path=root_path+'UCSF/UCSF-PDGM/Metadata/UCSF-PDGM-Pre-Post.xlsx'
data_id=pd.read_excel(xlsx_path)
ID=data_id['Patient'].tolist()
labels=[]

for ids in ID:
    mask_file=glob.glob(os.path.join(root_path , mri_data, mri_data + '-PDGM' '/Preprocessed/*/Baseline/',ids +'*tumor_segmentation.nii.gz'))
    #print(os.path.join(root_path , mri_data, mri_data + '-PDGM' '/Preprocessed/Post-Op/*/time01/',ids +'*seg_brats.nii.gz'))
    labels.append(mask_file[0])
    #print(mask_files)

labels=sorted(labels)
print(len(labels))

output_path = root_path + 'UCSF/UCSF-PDGM/Feature_extraction/Radiomics_features/'

In [31]:
print(len(labels))
labels

13


['/app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/100_0551/Baseline/100_0551_01_tumor_segmentation_brats_SRI24.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/100_1308/Baseline/100_1308_01_tumor_segmentation_brats_SRI24.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/100_296/Baseline/100_296_01_tumor_segmentation_brats_SRI24.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/100_302/Baseline/100_302_01_tumor_segmentation_brats_SRI24.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/100_4660/Baseline/100_4660_01_tumor_segmentation_brats_SRI24.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/100_5980/Baseline/100_5980_01_tumor_segmentation_brats_SRI24.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/100_719/Baseline/100_719_01_tum

## Specifiy tumor habitat

In [39]:
label_val = 4
# check for every database (Brats convention)
if label_val == 4:
    habitat = 'Enhancing'
elif label_val == 2:
    habitat = 'Edema'
elif label_val == 1:
    habitat = 'Necrosis'
elif label_val == 0:
    habitat = 'Habitat'
habitat

'Enhancing'

In [40]:
output_path2=os.path.join(root_path, mri_data,'Feature_extraction/Feature_matrix/')
outname=output_path2 + time_point + '/' + mri_data + '_' + habitat +'_local_shape_pp2.xlsx'
print(outname)

/app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Feature_extraction/Feature_matrix/Baseline/xCures_Enhancing_local_shape_pp2.xlsx


## Run feature extractor

In [41]:
feature_mat=[]
ids = []
ls_stats = []

idx_df=[]

for im in list(labels[:]):
   
    ls_stats=[]
    nib_image = nib.load(im)
    mri_image = nib_image.get_fdata()
    print('mask labels ... ', np.unique(mri_image))
    id_label = os.path.basename((os.path.dirname(os.path.dirname(im))))
    #id_label = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(im)))))
    print('id_label (patient ID) ......... ',id_label)
    print('tumor habitat ......... ',habitat)
    time_point = os.path.basename((os.path.dirname(((im)))))
    #time_point = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname((im)))))
    print('time point ......... ',time_point)
    feature_folder=os.path.join(output_path, id_label,time_point,'shape_global')
    tmp_name =  os.path.join(feature_folder,id_label + '_' + habitat + '_topology_pp2' + '.npy') 
    print('tmp_name .npy .......',tmp_name)
    
    if not os.path.isdir(feature_folder):
        os.makedirs(feature_folder)
        print("Directory created.")
    
    mri_image_habitat = np.zeros((mri_image.shape))
    
    if label_val != 0 and label_val in mri_image:
        mri_image_habitat[mri_image==label_val] = 1 ### consider only specified label
    elif label_val == 0:
        mri_image_habitat[mri_image>0] = 1
        
    if label_val == 0 or label_val in mri_image:
        
#         if not os.path.isfile(tmp_name):
            mri_image_habitat=np.float64(mri_image_habitat)
            # sanitized_mask = toprad.convert_volume_into_mask(mri_image, merge_labels=[label_val])
            sanitized_mask = toprad.convert_volume_into_mask(mri_image_habitat, merge_labels=[1])
            features = toprad.compute_morphology_features(sanitized_mask)
            df = features.to_DF()
            topology_values=df.values.T
            np.save(tmp_name, topology_values)

            stats = df.agg({'curvedness': ['mean', 'median', 'var', 'skew', 'kurt'],
                   'sharpness': ['mean', 'median', 'var', 'skew', 'kurt'],
                   'shape_index': ['mean', 'median', 'var', 'skew', 'kurt'],
                   'total_curvature': ['mean', 'median', 'var', 'skew', 'kurt']})
            vals = stats.values
            matT = vals.T
            vect = matT.reshape(1,20)
            ls_stats.append(vect)
            print('-region prop 3D scikit-image in .......................', im)

            np_stats = np.squeeze(ls_stats)
            feat_vect=np_stats.reshape(1, len(np_stats))
            props = regionprops(mri_image_habitat.astype(int))

    #         if props[0].pixelcount<50:
    #             print('noysi_area')
    #             continue
            prop3d=list([props[0].area_convex,props[0].area_filled,props[0].extent,props[0].feret_diameter_max,
                        props[0].solidity])
            n_arr=np.array(prop3d)
            n_arr=n_arr.reshape((1,len(prop3d)))
            all_shape=[]
            all_shape=np.hstack((n_arr,feat_vect))
            feature_mat.append(all_shape)
            ids.append(id_label)
            idx_df.append(id_label)
#         else: 
            print('..... npy already exist ..... ')
        
    else:
        print('------------ no sub-region -----------------------')
        continue

mask labels ...  [0. 4.]
id_label (patient ID) .........  100_0551
tumor habitat .........  Enhancing
time point .........  Baseline
tmp_name .npy ....... /app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Feature_extraction/Radiomics_features/100_0551/Baseline/shape_global/100_0551_Enhancing_topology_pp2.npy


2024-07-05 13:19:48,364 DEBUG convert_volume_into_mask  Found Labels: [0.0, 1.0]
2024-07-05 13:19:48,390 DEBUG convert_volume_into_mask  Setting mask label 0.0 to 0
2024-07-05 13:19:48,413 DEBUG convert_volume_into_mask  Setting mask label 1.0 to 1
2024-07-05 13:19:48,641 DEBUG compute_morphology_features  mask type is float64
2024-07-05 13:19:48,642 INFO compute_morphology_features  Starting Smoothing (iterations=3, sigma=3.0)
2024-07-05 13:19:48,642 DEBUG compute_morphology_features  Iteration 1: smoothing using sigma: 3.0
2024-07-05 13:19:48,643 DEBUG compute_morphology_features  smoothed_mri_mask type is float64
2024-07-05 13:19:48,644 DEBUG compute_morphology_features  Iteration 1: smoothing using sigma: 3.0
2024-07-05 13:19:49,124 DEBUG compute_morphology_features  smoothed_mri_mask type is float64
2024-07-05 13:19:49,125 DEBUG compute_morphology_features  Iteration 2: smoothing using sigma: 3.0
2024-07-05 13:19:49,613 DEBUG compute_morphology_features  smoothed_mri_mask type is 

-region prop 3D scikit-image in ....................... /app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/100_0551/Baseline/100_0551_01_tumor_segmentation_brats_SRI24.nii.gz
..... npy already exist ..... 
mask labels ...  [0. 1. 2. 4.]
id_label (patient ID) .........  100_1308
tumor habitat .........  Enhancing
time point .........  Baseline
tmp_name .npy ....... /app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Feature_extraction/Radiomics_features/100_1308/Baseline/shape_global/100_1308_Enhancing_topology_pp2.npy


2024-07-05 13:19:52,459 DEBUG convert_volume_into_mask  Found Labels: [0.0, 1.0]
2024-07-05 13:19:52,483 DEBUG convert_volume_into_mask  Setting mask label 0.0 to 0
2024-07-05 13:19:52,508 DEBUG convert_volume_into_mask  Setting mask label 1.0 to 1
2024-07-05 13:19:52,748 DEBUG compute_morphology_features  mask type is float64
2024-07-05 13:19:52,749 INFO compute_morphology_features  Starting Smoothing (iterations=3, sigma=3.0)
2024-07-05 13:19:52,749 DEBUG compute_morphology_features  Iteration 1: smoothing using sigma: 3.0
2024-07-05 13:19:52,750 DEBUG compute_morphology_features  smoothed_mri_mask type is float64
2024-07-05 13:19:52,751 DEBUG compute_morphology_features  Iteration 1: smoothing using sigma: 3.0
2024-07-05 13:19:53,271 DEBUG compute_morphology_features  smoothed_mri_mask type is float64
2024-07-05 13:19:53,272 DEBUG compute_morphology_features  Iteration 2: smoothing using sigma: 3.0
2024-07-05 13:19:53,792 DEBUG compute_morphology_features  smoothed_mri_mask type is 

-region prop 3D scikit-image in ....................... /app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/100_1308/Baseline/100_1308_01_tumor_segmentation_brats_SRI24.nii.gz
..... npy already exist ..... 
mask labels ...  [0. 1. 2. 4.]
id_label (patient ID) .........  100_296
tumor habitat .........  Enhancing
time point .........  Baseline
tmp_name .npy ....... /app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Feature_extraction/Radiomics_features/100_296/Baseline/shape_global/100_296_Enhancing_topology_pp2.npy


...

-region prop 3D scikit-image in ....................... /app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/102_911/Baseline/102_911_01_tumor_segmentation_brats_SRI24.nii.gz
..... npy already exist ..... 
mask labels ...  [0. 1. 2. 4. 5.]
id_label (patient ID) .........  103_637
tumor habitat .........  Enhancing
time point .........  Baseline
tmp_name .npy ....... /app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Feature_extraction/Radiomics_features/103_637/Baseline/shape_global/103_637_Enhancing_topology_pp2.npy


2024-07-05 13:21:13,722 DEBUG convert_volume_into_mask  Found Labels: [0.0, 1.0]
2024-07-05 13:21:13,750 DEBUG convert_volume_into_mask  Setting mask label 0.0 to 0
2024-07-05 13:21:13,777 DEBUG convert_volume_into_mask  Setting mask label 1.0 to 1
2024-07-05 13:21:14,034 DEBUG compute_morphology_features  mask type is float64
2024-07-05 13:21:14,035 INFO compute_morphology_features  Starting Smoothing (iterations=3, sigma=3.0)
2024-07-05 13:21:14,035 DEBUG compute_morphology_features  Iteration 1: smoothing using sigma: 3.0
2024-07-05 13:21:14,036 DEBUG compute_morphology_features  smoothed_mri_mask type is float64
2024-07-05 13:21:14,036 DEBUG compute_morphology_features  Iteration 1: smoothing using sigma: 3.0
2024-07-05 13:21:14,603 DEBUG compute_morphology_features  smoothed_mri_mask type is float64
2024-07-05 13:21:14,604 DEBUG compute_morphology_features  Iteration 2: smoothing using sigma: 3.0
2024-07-05 13:21:15,132 DEBUG compute_morphology_features  smoothed_mri_mask type is 

-region prop 3D scikit-image in ....................... /app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/103_637/Baseline/103_637_01_tumor_segmentation_brats_SRI24.nii.gz
..... npy already exist ..... 
mask labels ...  [0. 1. 2. 4. 5.]
id_label (patient ID) .........  103_737
tumor habitat .........  Enhancing
time point .........  Baseline
tmp_name .npy ....... /app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Feature_extraction/Radiomics_features/103_737/Baseline/shape_global/103_737_Enhancing_topology_pp2.npy


2024-07-05 13:21:17,726 DEBUG convert_volume_into_mask  Found Labels: [0.0, 1.0]
2024-07-05 13:21:17,753 DEBUG convert_volume_into_mask  Setting mask label 0.0 to 0
2024-07-05 13:21:17,779 DEBUG convert_volume_into_mask  Setting mask label 1.0 to 1
2024-07-05 13:21:18,018 DEBUG compute_morphology_features  mask type is float64
2024-07-05 13:21:18,019 INFO compute_morphology_features  Starting Smoothing (iterations=3, sigma=3.0)
2024-07-05 13:21:18,020 DEBUG compute_morphology_features  Iteration 1: smoothing using sigma: 3.0
2024-07-05 13:21:18,021 DEBUG compute_morphology_features  smoothed_mri_mask type is float64
2024-07-05 13:21:18,021 DEBUG compute_morphology_features  Iteration 1: smoothing using sigma: 3.0
2024-07-05 13:21:18,548 DEBUG compute_morphology_features  smoothed_mri_mask type is float64
2024-07-05 13:21:18,549 DEBUG compute_morphology_features  Iteration 2: smoothing using sigma: 3.0
2024-07-05 13:21:19,076 DEBUG compute_morphology_features  smoothed_mri_mask type is 

-region prop 3D scikit-image in ....................... /app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/Pre-op-Post-op/103_737/Baseline/103_737_01_tumor_segmentation_brats_SRI24.nii.gz
..... npy already exist ..... 


In [42]:
df = features.to_DF()

topology_values=df.values.T
print(topology_values.shape)
df

(8, 4402)


,shape_index,curvedness,sharpness,total_curvature,x,y,z,face_idx
0,0.501870,0.245965,0.219489,0.547475,147.996643,134.000000,46.000000,0
1,0.491226,0.161012,0.106558,0.326432,148.000000,134.000000,45.954468,1
2,0.503945,0.245965,0.219489,0.547475,148.000000,133.986069,46.000000,2
3,0.469903,0.121586,0.064715,0.254392,148.000000,134.000000,46.030468,3
4,0.617083,0.076513,0.014997,0.178431,147.959488,135.000000,45.000000,4
...,...,...,...,...,...,...,...,...
4397,-0.999992,0.026983,0.000000,0.076321,177.135056,127.000000,46.000000,4397
4398,0.702907,0.046381,0.003484,0.117158,177.074005,128.000000,42.000000,4398
4399,0.698355,0.039721,0.002628,0.099970,177.163162,128.000000,43.000000,4399
4400,0.698246,0.041363,0.002852,0.104093,177.174866,128.000000,44.000000,4400


In [43]:
names=[]
cl = stats.columns.values.tolist()
rw = stats.index.values.tolist()
for c in cl:
        for r in rw:
            names.append(r + '_' + c)
props_names=list(['area_convex','area_filled','extent','feret_diameter_max','solidity'])
    
props_names = props_names + names
len(props_names)

25

In [44]:
np.squeeze(feature_mat).shape
df_shape = pd.DataFrame(np.squeeze(feature_mat), index = ids, columns = props_names)
df_idx = pd.DataFrame(np.squeeze(idx_df), index = ids)

if not os.path.isdir(output_path2 + time_point):
        os.makedirs(output_path2 + time_point)
        print("Directory created.")

with pd.ExcelWriter(outname) as writer:
                df_shape.to_excel(writer, sheet_name="FeatureMatrix",index_label='ID')
                df_idx.to_excel(writer, sheet_name="df_idx",index_label='list_id')

print('saved in ...... ', outname)
print('patients', np.squeeze(feature_mat).shape)

saved in ......  /app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Feature_extraction/Feature_matrix/Baseline/xCures_Enhancing_local_shape_pp2.xlsx
patients (13, 25)


In [30]:
#         prop3d=list([props[0].pixelcount,props[0].area,props[0].area_convex,props[0].area_filled,props[0].axis_major_length,
#                     props[0].axis_minor_length,props[0].equivalent_diameter_area,props[0].euler_number,
#                     props[0].extent,props[0].feret_diameter_max,n_inertia[0],n_inertia[1],n_inertia[2],
#                     props[0].solidity])

#     props_names=list(['pixelcount','area','area_convex','area_filled','axis_major_length',
#                 'axis_minor_length','equivalent_diameter_area','euler_number','extent',
#                 'feret_diameter_max','eigvals_inertia_0','eigvals_inertia_1','eigvals_inertia_2',
#                 'solidity'])